<h3>ФБ-21 Князян Кирило</h3>
<h1><center><b> Лабораторна робота №2 </b></center></h1>
<h2><center> Наука про дані: підготовчий етап </center></h2>

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
import urllib.request
print("Setup Complete")

Setup Complete


In [14]:
if not os.path.exists('lab2_data'):
    os.makedirs('lab2_data')
def getting_data(i):
    filename_pattern = f"vhi_id_{i}"
    file_check = [el for el in os.listdir('lab2_data') if el.startswith(filename_pattern)]
    if file_check:
        return
    url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean'
    vhi_url = urllib.request.urlopen(url)
    now = datetime.datetime.now()
    date_and_time = now.strftime("%d-%m-%Y-%H-%M-%S")
    filename = f'vhi_id_{i}_{date_and_time}.csv'
    file_path = os.path.join('lab2_data', filename)
    with open(file_path, 'wb') as out:
        out.write(vhi_url.read())
    return f"File {file_path} downloaded"
for i in range(1, 28):
    getting_data(i)

Зчитати завантажені текстові файли у фрейм...

In [15]:
def data_frame(path):
    files = [el for el in os.listdir(path) if el.endswith('.csv')]
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    dfs = []
    for file in files:
        file_path = os.path.join(path, file)
        i = int(file.split('_')[2])
        if i == 12 or i == 20:
            continue
        df = pd.read_csv(file_path, header = 1, names = headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop('empty', axis=1)
        df.at[0, 'Year'] = df.at[0, 'Year'][9:]
        df = df.drop(df.index[-1])
        df['Year'] = df['Year'].astype(int)
        df['Week'] = df['Week'].astype(int)
        df['area'] = i
        dfs.append(df)
    df_r = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
    return df_r
df = data_frame('lab2_data')
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982     1  0.059  258.24  51.11  48.78  49.95    10
1      1982     2  0.063  261.53  55.89  38.20  47.04    10
2      1982     3  0.063  263.45  57.30  32.69  44.99    10
3      1982     4  0.061  265.10  53.96  28.62  41.29    10
4      1982     5  0.058  266.42  46.87  28.57  37.72    10
...     ...   ...    ...     ...    ...    ...    ...   ...
49445  2020    48  0.159  277.21  66.52  16.25  41.37     9
49446  2020    49  0.150  275.06  65.68  19.44  42.55     9
49447  2020    50  0.144  274.00  66.17  18.69  42.43     9
49448  2020    51  0.138  272.73  66.79  19.59  43.19     9
49449  2020    52  0.132  271.44  67.24  21.90  44.57     9

[49450 rows x 8 columns]


Реалізувати процедуру, яка змінить індекси областей...

In [16]:
province_indexes = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df['area'].replace(province_indexes, inplace = True)
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982     1  0.059  258.24  51.11  48.78  49.95    21
1      1982     2  0.063  261.53  55.89  38.20  47.04    21
2      1982     3  0.063  263.45  57.30  32.69  44.99    21
3      1982     4  0.061  265.10  53.96  28.62  41.29    21
4      1982     5  0.058  266.42  46.87  28.57  37.72    21
...     ...   ...    ...     ...    ...    ...    ...   ...
49445  2020    48  0.159  277.21  66.52  16.25  41.37    20
49446  2020    49  0.150  275.06  65.68  19.44  42.55    20
49447  2020    50  0.144  274.00  66.17  18.69  42.43    20
49448  2020    51  0.138  272.73  66.79  19.59  43.19    20
49449  2020    52  0.132  271.44  67.24  21.90  44.57    20

[49450 rows x 8 columns]


Ряд VHI для області за вказаний рік, пошук екстремумів (min та 
max;

In [5]:
def vhi_year(pro_index, year):
    if pro_index not in range(1, 26):
        return f"There is no province with index: {pro_index}"
    elif not df['Year'].isin([year]).any():
        return f"There is no information for province with index: {pro_index} in {year}"
    print(f'Data for province {pro_index} in {year}:')
    vhi_for_year = df[(df['area'] == pro_index) & (df['Year'] == year)][['Week', 'VHI']]
    print(vhi_for_year.to_string(index=False))
    min = vhi_for_year['VHI'].min()
    max = vhi_for_year['VHI'].max()
    print(f"Min VHI value is: {min}")
    print(f"Max VHI value is: {max}")
index = 5
year = 2000
vhi_year(index, year)

Data for province 5 in 2000:
 Week   VHI
    1 27.46
    2 30.88
    3 33.00
    4 36.72
    5 39.12
    6 40.34
    7 39.48
    8 39.81
    9 42.53
   10 44.97
   11 45.46
   12 47.78
   13 49.41
   14 51.16
   15 53.47
   16 57.49
   17 62.69
   18 66.30
   19 66.20
   20 64.55
   21 60.24
   22 55.93
   23 52.18
   24 49.62
   25 48.66
   26 46.97
   27 45.74
   28 46.53
   29 50.22
   30 53.37
   31 53.90
   32 54.14
   33 53.88
   34 53.31
   35 55.92
   36 58.65
   37 60.19
   38 61.07
   39 58.87
   40 51.27
   41 45.46
   42 41.35
   43 40.10
   44 36.37
   45 34.88
   46 35.76
   47 37.61
   48 39.92
   49 42.05
   50 43.06
   51 43.44
   52 40.43
Min VHI value is: 27.46
Max VHI value is: 66.3


Ряд VHI за вказаний діапазон років для вказаних областей

In [21]:
def vhi_years(pro_indexes, start_year, end_year):
    if any(i not in range(1, 26) for i in pro_indexes):
        return f"Incorrect province index or indexes, valid range from 1 to 25 inclusive"
    elif start_year > end_year:
        return f"Incorrect year range: {start_year}-{end_year}"
    vhi_for_years = df[(df['area'].isin(pro_indexes)) & (df['Year'] >= start_year) & (df['Year'] <= end_year)][['Year', 'VHI', 'area']]
    return vhi_for_years
indexes = [1, 4, 6, 7, 10, 19, 25]
start_year = 2000
end_year = 2020
vhi_for_years = vhi_years(indexes, start_year, end_year)
print(vhi_for_years)
vhi_for_years = vhi_for_years.groupby('Year')['VHI'].mean()
print(vhi_for_years)

       Year    VHI  area
4862   2000  43.87    10
4863   2000  45.92    10
4864   2000  44.66    10
4865   2000  41.77    10
4866   2000  41.42    10
...     ...    ...   ...
47467  2020  36.12    19
47468  2020  37.49    19
47469  2020  36.90    19
47470  2020  38.88    19
47471  2020  42.54    19

[7504 rows x 3 columns]
Year
2000    43.620330
2001    52.281099
2002    50.399918
2003    49.692129
2004    61.808866
2005    50.885098
2006    48.183984
2007    41.410742
2008    48.409753
2009    43.363077
2010    46.323901
2011    45.004835
2012    40.608956
2013    46.744698
2014    47.136538
2015    42.722473
2016    50.091209
2017    45.299670
2018    44.487143
2019    42.702033
2020    42.270275
Name: VHI, dtype: float64


Виявити роки, протягом яких екстремальні посухи торкнулися 
більше вказаного відсотка областей по Україі... Аналогічно для помірних посухі

In [7]:
def extreme_droughts(pro_percent, pro_indexes, start_year, end_year):
    vhi_for_years = vhi_years(pro_indexes, start_year, end_year)
    if isinstance(vhi_for_years, str):
        return vhi_for_years
    pro_amount = len(pro_indexes)
    pro_amount_for_drougths = round(pro_amount*(pro_percent/100))
    extreme_drought_years = {}
    for year in range(start_year, end_year+1):
        vhi_year = vhi_for_years[vhi_for_years['Year'] == year]
        extreme_drought_areas = vhi_year[vhi_year['VHI'] < 15]['area'].nunique()
        if extreme_drought_areas > pro_amount_for_drougths:
            extreme_drought_years[year] = extreme_drought_areas
    for year, amount in extreme_drought_years.items():
        print(f"Amount of provinces with extreme drought in {year}: {amount}")

def moderate_droughts(pro_percent, pro_indexes, start_year, end_year):
    vhi_for_years = vhi_years(pro_indexes, start_year, end_year)
    if isinstance(vhi_for_years, str):
        return vhi_for_years
    pro_amount = len(pro_indexes)
    pro_amount_for_drougths = round(pro_amount*(pro_percent/100))
    moderate_droughts_years = {}
    for year in range(start_year, end_year+1):
        vhi_year = vhi_for_years[vhi_for_years['Year'] == year]
        moderate_drought_areas = vhi_year[vhi_year['VHI'] < 35]['area'].nunique()
        if moderate_drought_areas > pro_amount_for_drougths:
            moderate_droughts_years[year] = moderate_drought_areas
    for year, amount in moderate_droughts_years.items():
        print(f"Amount of provinces with moderate drought in {year}: {amount}")

pro_percent = 20
indexes = [i for i in range(1, 26)]
start_year = 2000
end_year = 2020
extreme_droughts(pro_percent, indexes, start_year, end_year)
moderate_droughts(pro_percent, indexes, start_year, end_year)

Amount of provinces with moderate drought in 2000: 25
Amount of provinces with moderate drought in 2001: 10
Amount of provinces with moderate drought in 2002: 12
Amount of provinces with moderate drought in 2003: 13
Amount of provinces with moderate drought in 2005: 7
Amount of provinces with moderate drought in 2006: 12
Amount of provinces with moderate drought in 2007: 15
Amount of provinces with moderate drought in 2008: 15
Amount of provinces with moderate drought in 2009: 14
Amount of provinces with moderate drought in 2010: 13
Amount of provinces with moderate drought in 2011: 16
Amount of provinces with moderate drought in 2012: 12
Amount of provinces with moderate drought in 2013: 16
Amount of provinces with moderate drought in 2014: 11
Amount of provinces with moderate drought in 2015: 23
Amount of provinces with moderate drought in 2016: 15
Amount of provinces with moderate drought in 2017: 11
Amount of provinces with moderate drought in 2018: 15
Amount of provinces with mode